# Example for EVICORD sampler

In [ ]:
library(EVICORD)
library(ggplot2)
library(dplyr)
setwd("./EVICORD/example")

## Set up: allele count & number of allele pairs

In [ ]:
ac = 4 #allele count
npairs = choose(ac,2) #number of allele pairs

## Simulate data
Generate simulated pairwise recombination distances for 200 IBD variants, 200 recurrent variants (with partition 1:3).

In [ ]:
#simulate pairwise recombination distances for IBD variants
nibd = 200 #number of variants
tibd = rgamma(nibd,shape=10,rate=0.1) #sample nibd TMRCAs

ibddat = tbl_df(data.frame()) #place to store IBD variant recomb distances

for (i in 1:nibd){ #for each variant
    distsL = rexp(npairs,rate=tibd[i]/50) #recomb dists left side
    distsR = rexp(npairs,rate=tibd[i]/50) #recomb dists right side
    vardat = cbind(rep(i,npairs),distsL,distsR)
    ibddat = rbind(ibddat,vardat)
}

colnames(ibddat) = c("varID","distL","distR")
ibddat = tbl_df(ibddat) %>% mutate(type="ibd")

#simulate pairwise recombination distances for recurrent variants
#use partition 1:3 (recurrent variant comprised of a singleton & a tripleton)
nrec = 200 #number of variants
trec_rec = rgamma(nrec,shape=40,rate=0.08) #sample nibd TMRCAs for recurrent allele pairs
trec_ibd = rgamma(nrec,shape=8,rate=0.1) #sample nibd TMRCAs for IBD allele pairs

recdat = tbl_df(data.frame()) #place to store recurrent variant recomb distances

for (i in 1:nrec){ #for each variant
    #sample distances of rec allele pairs
    distsLrec = rexp(3,rate=trec_rec[i]/50) #in 1:3 partition there are 1x3=3 recurrent allele pairs
    distsRrec = rexp(3,rate=trec_rec[i]/50)
    
    #sample distances of ibd allele pairs
    distsLibd = rexp(3,rate=trec_ibd[i]/50) #in 1:3 partition there are choose(3,2)=3 IBD allele pairs
    distsRibd = rexp(3,rate=trec_ibd[i]/50)
    
    vardat = cbind(rep(i+nibd,npairs),c(distsLrec,distsLibd),c(distsRrec,distsRibd))
    recdat = rbind(recdat,vardat)
}
colnames(recdat) = c("varID","distL","distR")
recdat = tbl_df(recdat) %>% mutate(type="rec")

#combine recomb distances rec & IBD
alldat = bind_rows(ibddat,recdat)

Visualize simulated TMRCAs.

In [ ]:
#plot density of TMRCAs
tmrca = cbind(tibd,trec_ibd,trec_rec) %>% tbl_df() %>% tidyr::gather(key="pairType",value="t",1:3)
ggplot(tmrca) + geom_density(aes(x=t,color=pairType))

Visualize simulated recombination distances:

In [ ]:
#plot density of pairwise recombination distances
ggplot(alldat) + geom_density(aes(x=distL,color=type))

Save simulated recombination distances to file.

In [ ]:
write.table(alldat[,1:3],"sim_dists_ex.txt",quote=F,sep="\t",row.names=F)

## Run sampler

In [ ]:
#load distances from file (3 cols: varID, distL, distR)
dists = load_dists("sim_dists_ex.txt",header=T,ac=4)

#run sampler
sampler(dists,ac=4,alphaRec=40,betaRec=0.1,alphaIBD=10,alpha0=1,beta0=10,piPriors=c(0.4,0.4,0.2),niter=100,outfile="gibbs_sampler_out.rds")


## Visualize sampler output

In [ ]:
viz_res("gibbs_sampler_out.rds",ac=4,tf=5,outfile="gibbs_sampler_out_viz.pdf")